<a href="https://colab.research.google.com/github/DanielSCarmo/CogniLens_Imersao_IA/blob/main/Cogni_Lens_%C3%93culos_Inteligente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto/Protótipo
Nosso projeto consiste no desenvolvimento de um protótipo inicial de óculos integrado com a inteligência artificial Gemini. O dispositivo incluirá uma câmera e um fone de ouvido para a transmissão e recebimento de informações da IA. Optamos por utilizar o Code Visual Studio para o desenvolvimento do código, uma vez que o Colab apresentou restrições no acesso à câmera e ao áudio. Este enfoque visa simplificar ao máximo a implementação inicial do projeto.
**\ O código não roda no Colab! /**

# Baixar as bibliotecas usadas

In [ ]:
!pip install Opencv-python
!pip install SpeechRecognition
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0
!pip install pyaudio
!pip install pyttsx3
!pip install matplotlib
!pip install tk
!pip install pydub

# Importando as bibliotecas

In [2]:
import cv2
import numpy as np
import pyaudio
import wave
from pathlib import Path
import hashlib
import google.generativeai as genai
import speech_recognition as sr
import tkinter as tk
import pyttsx3
import threading

# Esperando o usuario falar Ok Gemini para começar o comando

In [14]:
# Há duas chaves api pois foi feito em grupo
from google.colab import userdata
api_key = userdata.get('Chave_secreta')
genai.configure(api_key=api_key)

api_key = userdata.get('Chave_secreta2')
genai.configure(api_key=api_key)

In [ ]:
# Como é apenas um protótipo, substituimos o OK Gemini por apertar a tecla Q
# que vai começar a executar o código abaixo que é o comando para o Gemini

# Inicializa o reconhecedor de fala
reconhecedor = sr.Recognizer()

# Variável para controlar a execução da transcrição
executando = True

# Função para transcrever o áudio e atualizar o texto na janela
def transcrever_audio():
    global executando
    with sr.Microphone() as source:
        print("Diga algo!")
        # Escuta o áudio do microfone
        audio = reconhecedor.listen(source, phrase_time_limit=10)  # Limita o tempo de escuta a 5 segundos

        try:
            # Usa o Google Speech Recognition para transcrever o áudio em texto
            texto = reconhecedor.recognize_google(audio, language="pt-BR")  # Define o idioma como português do Brasil
            if texto:
                resultado_label.config(text="Você disse: " + texto)
        except sr.UnknownValueError:
            resultado_label.config(text="Não foi possível entender o áudio")
        except sr.RequestError as e:
            resultado_label.config(text="Erro no serviço de reconhecimento de fala; {0}".format(e))

        # Agendar a próxima transcrição após 1000 milissegundos, se a variável executando for True
        if executando:
            janela.after(1000, transcrever_audio)

# Função para encerrar o programa
def encerrar_programa(event):
    global executando
    if event.keysym == 'q':
        executando = False
        janela.destroy()

# Cria a janela
janela = tk.Tk()
janela.title("Transcrição de Áudio")

# Cria um rótulo para exibir o resultado da transcrição
resultado_label = tk.Label(janela, text="")
resultado_label.pack()

# Inicia a transcrição do áudio em uma thread separada
threading.Thread(target=transcrever_audio).start()

# Associa a função de encerramento do programa ao pressionar uma tecla
janela.bind('<Key>', encerrar_programa)

# Inicia o loop principal da janela
janela.mainloop()

# Começar o comando para o Gemini

In [ ]:
# Inicializa o gravador de vídeo
frame_width = 640
frame_height = 480
out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), 20, (frame_width, frame_height))

# Inicializa o gravador de áudio
audio = pyaudio.PyAudio()
stream = audio.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=1024)
frames = []

# Captura de vídeo
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    if ret:
        # Escreve o frame no arquivo de vídeo
        out.write(frame)
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
    # Captura o áudio
    data = stream.read(1024)
    frames.append(data)

# Libera os recursos
cap.release()
out.release()
cv2.destroyAllWindows()
stream.stop_stream()
stream.close()
audio.terminate()

# Salva o áudio em um arquivo WAV
wf = wave.open("output_audio.wav", 'wb')
wf.setnchannels(1)
wf.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
wf.setframerate(44100)
wf.writeframes(b''.join(frames))
wf.close()

In [ ]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""

genai.configure(api_key=api_key)

# Set up the model
generation_config = {
  "temperature": 0.57,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_ONLY_HIGH"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_ONLY_HIGH"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_ONLY_HIGH"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

def extract_video_frames(pathname: str) -> list:
  # See https://github.com/google-gemini/cookbook/blob/main/quickstarts/Video.ipynb
  return []

prompt_parts = [
  "\nListen to the audio, try to understand what is being said and respond to the audio command:\n",
  genai.upload_file("output_audio.wav"),
  "\nrespond to the audio using the following video as a parameter:\n",
  *extract_video_frames("output.avi"),
  "\nrespond in Portuguese (Brazil)",
]

response = model.generate_content(prompt_parts)
print(response.text)


# Resposta em áudio

In [ ]:
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# Exemplo de uso
#response = "Olá, eu sou o Gemini. Como posso ajudá-lo hoje?"
#speak(response)
speak(response.text)